In [1]:
pip install -r "requirement.txt"


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import sqlite3
import pandas as pd
db_file = "Fund.db"
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'TSLA', 'NVDA', 'PYPL', 'ADBE', 'INTC']#tickers contenus dans MAIN_DATA_FRAME

start_date = "01/09/2022"#toute la periode start date (pour tout telecharger dun coup) + 3 mois avant
end_date = "31/12/2024"#pareil

## 1. Creation et peuplement de la base

In [2]:
from base_builder import DatabaseBuilder

In [3]:
db_manager = DatabaseBuilder(db_file)
db_manager.create_tables()
db_manager.insert_clients_data(12)
db_manager.insert_managers_data(3)

Tables created successfully.
12 clients insérés avec succès dans la table 'Clients'.
3 managers insérés avec succès dans la table 'Managers'.


In [ ]:
db_manager.get_investment_amount_by_risk_type()
db_manager.insert_initial_cash_portfolios(start_date)


Résumé des investissements par risk type:
--------------------------------------------------
Risk Type: HY_EQUITY
Nombre de clients: 1
Montant total investi: 8,079,414.21 €
--------------------------------------------------
Risk Type: LOW_RISK
Nombre de clients: 4
Montant total investi: 11,580,832.92 €
--------------------------------------------------
Risk Type: LOW_TURNOVER
Nombre de clients: 7
Montant total investi: 30,227,108.91 €
--------------------------------------------------
Montants initiaux en cash insérés avec succès dans la table Portfolios.


## 2. Collecte et traitement des datas

In [5]:
from data_collector import GetData

In [6]:
gdata = GetData(tickers, start_date, end_date)
gdata.main_data_frame()#on cree le dataframe avec TOUS les actifs TOUTE la periode
#il manquerait le traitement des datas

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,IMPORT_DATE,PRICE,TICKER,SECTOR
0,2023-01-03,123.632530,AAPL,Technology
1,2023-01-04,124.907707,AAPL,Technology
2,2023-01-05,123.583092,AAPL,Technology
3,2023-01-06,128.130219,AAPL,Technology
4,2023-01-09,128.654144,AAPL,Technology
...,...,...,...,...
5005,2024-12-23,20.200001,INTC,Technology
5006,2024-12-24,20.400000,INTC,Technology
5007,2024-12-26,20.440001,INTC,Technology
5008,2024-12-27,20.299999,INTC,Technology


# 3. Periode avant t = 0 (Periode de "test")

On peuple Products avec des trois mois de donnees avant la periode d'evaluation du fonds (qui commence le 01/01/2023)

In [ ]:
from base_update import BaseUpdate 
sub_start_date = "01/09/2022"#chaque semaine, les strat lancent la fct et definissent les sub périodes
#de cette maniere, products ne contient que la data jusqu'à la date t (no forward looking sur nos strats)
sub_end_date = "29/12/2022"

In [ ]:
B_udt = BaseUpdate(tickers, sub_start_date, sub_end_date, db_file)
B_udt.update_products()

### Initialisation des portefeuilles en fonction du cash initial

Pour la strategie HY equity, on prend les 5 tickers les plus performants de la periode (allant du 01/09/2022 au 30/12/2022)

In [ ]:
B_udt.initialisation_portefeuille_HY()

# 4. Execution des strategies

In [ ]:
from strategies import RunAllStrat
start_date2="01/01/2023"
end_date2="01/10/2023"
b=RunAllStrat(db_file,start_date2,end_date2,tickers)
b.strategy_two(end_date2)

In [ ]:
import sqlite3

db_path = "Fund.db"  # adapte si ton fichier a un chemin différent

create_history_table_query = """
CREATE TABLE IF NOT EXISTS Portfolio_History (
    HISTORY_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    MANAGER_ID INTEGER NOT NULL,
    TICKER TEXT NOT NULL,
    QUANTITY INTEGER NOT NULL,
    DATE_SNAPSHOT DATE NOT NULL,
    FOREIGN KEY(MANAGER_ID) REFERENCES Managers(MANAGER_ID) ON DELETE CASCADE
);
"""

conn = sqlite3.connect(db_path)
cursor = conn.cursor()
cursor.execute(create_history_table_query)
conn.commit()
conn.close()

print("✅ Table Portfolio_History créée avec succès.")


In [ ]:
def rebuild_portfolio_history_by_manager(self, manager_id):
    try:
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        # 1. Composition initiale : actifs dans Portfolios pour ce manager
        cursor.execute("""
            SELECT TICKER, QUANTITY
            FROM Portfolios
            WHERE MANAGER_ID = ?
        """, (manager_id,))
        initial_assets = cursor.fetchall()
        portfolio = {ticker: qty for ticker, qty in initial_assets}

        # 2. Récupérer tous les deals du manager, triés par date
        cursor.execute("""
            SELECT EXECUTION_DATE, TICKER, TRADE_TYPE, QUANTITY
            FROM Deals
            WHERE MANAGER_ID = ?
            ORDER BY EXECUTION_DATE ASC
        """, (manager_id,))
        deals = cursor.fetchall()

        # 3. Appliquer chaque deal et sauvegarder un snapshot
        for deal in deals:
            date, ticker, trade_type, quantity = deal

            if trade_type == 'Buy':
                portfolio[ticker] = portfolio.get(ticker, 0) + quantity
            elif trade_type == 'Sell':
                portfolio[ticker] = max(portfolio.get(ticker, 0) - quantity, 0)

            # Retirer les actifs avec 0
            portfolio = {k: v for k, v in portfolio.items() if v > 0}

            # 4. Insérer un snapshot complet pour cette date
            for tick, qty in portfolio.items():
                cursor.execute("""
                    INSERT INTO Portfolio_History (MANAGER_ID, TICKER, QUANTITY, DATE_SNAPSHOT)
                    VALUES (?, ?, ?, ?)
                """, (manager_id, tick, qty, date))

        conn.commit()
        print(f"📈 Historique reconstruit pour le manager {manager_id}")
    except sqlite3.Error as e:
        print(f"❌ Erreur SQLite : {e}")
    finally:
        if conn:
            conn.close()


In [13]:
db_manager.rebuild_portfolio_history_by_manager(manager_id=1)


📈 Historique reconstruit pour le manager 1
